# Imports and utilities

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from shared.snowflake.client import SnowflakeClient

client = SnowflakeClient()

In [4]:
session = client.get_snowpark_session()

In [5]:
import logging

# Configure logging to show ALL messages (DEBUG and above)
logging.basicConfig(
    level=logging.DEBUG,  # Set to DEBUG to see everything
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    force=True  # Force reconfiguration if logging was already configured
)

# Or if you want to see messages in Jupyter Notebook output:
logging.getLogger().setLevel(logging.DEBUG)

In [6]:
# Transform service dependencies
from app.services.transform_service import TransformService
from app.models.transform import (
    TransformConstraints,
    TransformationType,
    TransformRequest,
    Recipe,
)

# Snowflake example usage

In [37]:
client.is_connected()

{'ok': True, 'version': '9.39.2'}

In [38]:
query = """
    SELECT 
        ID,
        NAME,
        INGREDIENTS,
        STEPS,
        MINUTES,
    FROM NUTRIRAG_PROJECT.RAW.RAW_RECIPES_10K
    LIMIT 1
"""

result_df = client.execute(query, fetch="one")

# Transform service example usage

In [7]:
transform_service = TransformService(session)

### Input definition

In [9]:
ingredients = [
    "crabmeat",
    "cream cheese",
    "green onions",
    "garlic salt",
    "refrigerated crescent dinner rolls",
    "egg yolk",
    "water",
    "sesame seeds",
    "sweet and sour sauce",
]
steps = [
    "heat over to 375 degrees",
    "spray large cookie sheet with non-stick cooking spray",
    "in small bowl , combine crabmeat , cream cheese , onions and garlic salt and mix well",
    "unroll both cans of dough",
    "separate into 16 triangles",
    "cut each triangle in half lengthwise to make 32 triangles",
    "place 1 teaspoon crab mixture on center of each triangle about 1 inch from short side of triangle",
    "fold short ends of each triangle over filling",
    "pinch sides to seal",
    "roll up",
    "place on sprayed cookie sheet",
    "in small bowl , combine egg yolk and water and mix well",
    "brush egg mixture over snacks",
    "sprinkle with sesame seed",
    "bake at 375 degrees for 15 to 20 minutes or until golden brown",
    "serve warn snacks with sweet-and-sour sauce",
]

recipe = Recipe(
    id=94947,
    name="crab filled crescent snacks",
    serving_size=1,
    servings=4,
    ingredients=ingredients,
    quantity_ingredients=["1"] * len(ingredients),
    minutes=70.0,
    steps=steps,
    health_score=57
)
ingredients_to_remove = ["cream cheese"]

constraints = TransformConstraints(
    transformation=TransformationType.SUBSTITUTION,
    no_lactose=False,
    no_gluten=False,
    no_nuts=False,
    vegetarian=False,
    vegan=False,
    increase_protein=False,
    decrease_sugar=False,
    decrease_protein=False,
    decrease_carbs=False,
    decrease_calories=False,
    decrease_sodium=False,
)
# En principe, on récupère la request en entrée. La définition est juste pour l'exemple ici.
request = TransformRequest(
    recipe=recipe,
    ingredients_to_remove=ingredients_to_remove,
    constraints=constraints,
)
# Pour éviter de refaire les requêtes SQL sur les ingrédients de la recipe
cache = {
    "crabmeat": {
        "name": "crabapples raw",
        "matched_ingredient": "crabmeat",
        "protein": 0.4,
        "saturated_fats": 0.048,
        "fat": 0.3,
        "carbs": 19.95,
        "sodium": 1.0,
        "fiber": 0.0,
        "sugar": 0.0,
        "calories": 76.0,
    },
    "cream cheese": {
        "name": "cheese cream fat free",
        "matched_ingredient": "cream cheese",
        "protein": 15.69,
        "saturated_fats": 0.644,
        "fat": 1.0,
        "carbs": 7.66,
        "sodium": 702.0,
        "fiber": 0.0,
        "sugar": 5.48,
        "calories": 105.0,
    },
    "green onions": {
        "name": "onions frz whl ckd bld drnd wsalt",
        "matched_ingredient": "green onions",
        "protein": 0.71,
        "saturated_fats": 0.009000000000000001,
        "fat": 0.05,
        "carbs": 6.11,
        "sodium": 244.0,
        "fiber": 1.4,
        "sugar": 2.9,
        "calories": 26.0,
    },
    "garlic salt": {
        "name": "garlic  single clove  kashmir",
        "matched_ingredient": "garlic salt",
        "protein": 6.12,
        "saturated_fats": 0.03476,
        "fat": 0.16,
        "carbs": 23.46,
        "sodium": 8.87,
        "fiber": 4.01,
        "sugar": 3.73,
        "calories": 124.99700000000001,
    },
    "refrigerated crescent dinner rolls": {
        "name": "rolls dinner egg",
        "matched_ingredient": "refrigerated crescent dinner rolls",
        "protein": 9.5,
        "saturated_fats": 1.577,
        "fat": 6.4,
        "carbs": 52.0,
        "sodium": 566.0,
        "fiber": 3.7,
        "sugar": 4.3,
        "calories": 307.0,
    },
    "egg yolk": {
        "name": "egg yolk raw frz salted past",
        "matched_ingredient": "egg yolk",
        "protein": 14.07,
        "saturated_fats": 7.159,
        "fat": 22.93,
        "carbs": 1.77,
        "sodium": 3487.0,
        "fiber": 0.0,
        "sugar": 0.07,
        "calories": 275.0,
    },
    "water": {
        "name": "water melon  pale green",
        "matched_ingredient": "water",
        "protein": 0.59,
        "saturated_fats": 0.05033,
        "fat": 0.16,
        "carbs": 3.02,
        "sodium": 1.62,
        "fiber": 0.78,
        "sugar": 3.0,
        "calories": 16.73,
    },
    "sesame seeds": {
        "name": "sesame seeds whl rstdtstd",
        "matched_ingredient": "sesame seeds",
        "protein": 16.96,
        "saturated_fats": 6.722,
        "fat": 48.0,
        "carbs": 25.74,
        "sodium": 11.0,
        "fiber": 14.0,
        "sugar": 0.0,
        "calories": 565.0,
    },
    "sweet and sour sauce": {
        "name": "cherries sweet raw",
        "matched_ingredient": "sweet and sour sauce",
        "protein": 1.06,
        "saturated_fats": 0.038,
        "fat": 0.2,
        "carbs": 16.01,
        "sodium": 0.0,
        "fiber": 2.1,
        "sugar": 12.82,
        "calories": 63.0,
    },
    "turkey  white  rotisserie  deli cut": {
        "name": "turkey  white  rotisserie  deli cut",
        "matched_ingredient": "turkey  white  rotisserie  deli cut",
        "protein": 13.5,
        "saturated_fats": 0.11800000000000001,
        "fat": 3.0,
        "carbs": 7.7,
        "sodium": 1200.0,
        "fiber": 0.4,
        "sugar": 4.0,
        "calories": 112.0,
    },
}
transform_service.ingredients_cache = cache

### Run Transform service

In [10]:
result = transform_service.transform(
    recipe=request.recipe,
    ingredients_to_remove=request.ingredients_to_remove,
    constraints=request.constraints,
)

2026-01-10 23:22:32,968 - root - INFO - Success: Step 1 finished (Ingredients to remove has been found).
2026-01-10 23:22:32,972 - root - INFO - Substitution: Looking for matched ingredients.


2026-01-10 23:22:34,849 - root - INFO - Success: PCA ingredients coordinates successfully loaded.
2026-01-10 23:22:34,923 - snowflake.connector.cursor - DEBUG - executing SQL/command
2026-01-10 23:22:34,931 - snowflake.connector.cursor - INFO - query: [SELECT DISTINCT "DESCRIP", "PROTEIN_G", "SATURATED_FATS_G", "FAT_G", "CARB_G", "...]
2026-01-10 23:22:34,933 - snowflake.connector.connection - DEBUG - sequence counter: 1
2026-01-10 23:22:34,936 - snowflake.connector.cursor - DEBUG - Request id: b8042815-4a18-48dc-8a29-3340ad292eed
2026-01-10 23:22:34,937 - snowflake.connector.cursor - DEBUG - running query [SELECT DISTINCT "DESCRIP", "PROTEIN_G", "SATURATED_FATS_G", "FAT_G", "CARB_G", "...]
2026-01-10 23:22:34,939 - snowflake.connector.cursor - DEBUG - is_file_transfer: True
2026-01-10 23:22:34,947 - snowflake.connector.connection - DEBUG - _cmd_query
2026-01-10 23:22:34,954 - snowflake.connector._query_context_cache - DEBUG - serialize_to_dict() called
2026-01-10 23:22:34,955 - snowfl

LLM: 3 adapted steps, 0 notes


2026-01-10 23:22:59,691 - root - INFO - Success: Step 3 finished for Substitution (LLM's adapted new_recipe steps successfully).
2026-01-10 23:22:59,746 - snowflake.connector.cursor - DEBUG - executing SQL/command
2026-01-10 23:22:59,748 - snowflake.connector.cursor - INFO - query: [SELECT  *  FROM NUTRIRAG_PROJECT.RAW.INGREDIENTS_MATCHING]
2026-01-10 23:22:59,750 - snowflake.connector.connection - DEBUG - sequence counter: 45
2026-01-10 23:22:59,751 - snowflake.connector.cursor - DEBUG - Request id: 9be8a0b7-a595-42a7-a571-d8098cebfed4
2026-01-10 23:22:59,753 - snowflake.connector.cursor - DEBUG - running query [SELECT  *  FROM NUTRIRAG_PROJECT.RAW.INGREDIENTS_MATCHING]
2026-01-10 23:22:59,754 - snowflake.connector.cursor - DEBUG - is_file_transfer: True
2026-01-10 23:22:59,755 - snowflake.connector.connection - DEBUG - _cmd_query
2026-01-10 23:22:59,759 - snowflake.connector._query_context_cache - DEBUG - serialize_to_dict() called
2026-01-10 23:22:59,800 - snowflake.connector._query

In [11]:
vars(result)

{'recipe': Recipe(id=94947, name='crab filled crescent snacks', serving_size=1.0, servings=4.0, health_score=0.0, ingredients=['crabmeat', 'sour cream reduced fat', 'green onions', 'garlic salt', 'refrigerated crescent dinner rolls', 'egg yolk', 'water', 'sesame seeds', 'sweet and sour sauce'], quantity_ingredients=['1', '1', '1', '1', '1', '1', '1', '1', '1'], minutes=70.0, steps=['Heat over to 375 degrees.', 'Spray large cookie sheet with non-stick cooking spray.', 'In small bowl, combine crabmeat, sour cream reduced fat, green onions and garlic salt and mix well.']),
 'original_name': 'crab filled crescent snacks',
 'transformed_name': 'crab filled crescent snacks',
 'substitutions': None,
 'nutrition_before': NutritionDelta(calories=0.0, protein_g=0.0, saturated_fats_g=0.0, fat_g=0.0, carb_g=0.0, fiber_g=0.0, sodium_mg=0.0, sugar_g=0.0, iron_mg=0.0, calcium_mg=0.0, magnesium_mg=0.0, potassium_mg=0.0, vitamin_c_mg=0.0, health_score=57.0),
 'nutrition_after': NutritionDelta(calories=

In [53]:
result.recipe.steps

['1. Heat over to 375 degrees.',
 '2. Spray large cookie sheet with non-stick cooking spray.',
 '3. In a small bowl, combine crabmeat, soybeans mature seeds sprouted ckd stir-fried wsalt, green onions, and garlic salt. Mix well.',
 '4. Unroll both cans of dough.',
 '5. Separate into 16 triangles.',
 '6. Cut each triangle in half lengthwise to make 32 triangles.',
 '7. Place 1 teaspoon crab mixture on the center of each triangle about 1 inch from the short side of the triangle.',
 '8. Fold short ends of each triangle over filling.',
 '9. Pinch sides to seal.',
 '10. Roll up.',
 '11. Place on sprayed cookie sheet.',
 '12. In a small bowl, combine egg yolk and water and mix well.',
 '13. Brush egg mixture over snacks.',
 '14. Sprinkle with sesame seeds.',
 '15. Bake at 375 degrees for 15 to 20 minutes or until golden brown. Note that the soybeans may provide a slightly different texture compared to the original cream cheese.',
 '16. Serve warm snacks with sweet-and-sour sauce.']

In [54]:
result.success

True

# Always close connexion before closing notebook

In [12]:
client.close()

2026-01-10 23:26:01,851 - snowflake.connector.connection - INFO - closed
2026-01-10 23:26:01,855 - snowflake.connector.telemetry - DEBUG - Closing telemetry client.
2026-01-10 23:26:01,943 - snowflake.connector.telemetry - DEBUG - Sending 1 logs to telemetry. Data is {'logs': [{'message': {'driver_type': 'PythonConnector', 'driver_version': '3.6.0', 'source': 'PythonConnector', 'type': 'client_imported_packages', 'value': "{'runpy', 'logging', 'dataclasses', 'zmq', 'calendar', 'errno', 'selectors', 'shutil', 'contextvars', 'bisect', 'copy', 'shlex', 'socketserver', 'jwt', 'cloudpickle', 'tomlkit', 'pathlib', 'profile', 'cProfile', 'pydevconsole', 'pydevd_tracing', 'cmd', 'fractions', 'yaml', 'numbers', 'enum', 'gc', 'comm', 'pyexpat', 'snowflake', 'platform', 'dis', 'pydevd', 'operator', 'glob', 'site', 'pytz', 'unittest', 'csv', 'decimal', 'quopri', 'locale', 'traceback', 'secrets', 'json', 'collections', 'copyreg', 'jupyter_client', 'tornado', 'urllib', 'jedi', 'appnope', 'pandas', '